In [2]:
import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
import keras.callbacks as kcall
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
train_dir = 'data/train-preprocess'

In [4]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

data/train-preprocess 0
data/train-preprocess/MEL 1113
data/train-preprocess/DF 115
data/train-preprocess/BKL 1099
data/train-preprocess/BCC 514
data/train-preprocess/VASC 142
data/train-preprocess/AKIEC 327
data/train-preprocess/NV 6705


In [7]:
from keras.preprocessing.image import ImageDataGenerator

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.25)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)



train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = target_size,       
        class_mode = 'categorical',
        subset="training",
        shuffle = True)

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = target_size,        
        class_mode = 'categorical',
        subset = "validation",
        shuffle = True)

Found 7515 images belonging to 7 classes.
Found 2500 images belonging to 7 classes.


In [8]:
## Intilizing variables
output_classes = 7
learning_rate = 0.001
img_width, img_height,channel = 600, 450, 3
batch_size = 32 
epochs = 1
resume_model = False
xception_weights = 'pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [9]:
if resume_model == False:
    model = Sequential()
 
    model.add(Xception(weights = xception_weights , include_top=False,pooling = 'avg'))

    model.add(Dense(units=output_classes, activation='softmax'))

    model.layers[0].trainable = True

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])

  ## model.load_weights('xception_weights_tf_dim_ordering_tf_kernels_notop.h5', by_name=True)


    for i, layer in enumerate(model.layers):
        print('Layer: ',i+1,' Name: ', layer.name)

Layer:  1  Name:  xception
Layer:  2  Name:  dense_1


In [ ]:
## Training only the newly added layer
if resume_model:
    model = load_model('skin-classification-Xception2.h5')
else: 
    history = model.fit_generator(train_generator,
        steps_per_epoch=400,
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps=100)

Epoch 1/1
  6/400 [..............................] - ETA: 24:40:57 - loss: 1.3639 - acc: 0.5365

In [ ]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Test Score: ', score[0])
print ('Test Accuracy: ',score[1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
filename = validation_generator.filenames
truth = validation_generator.classes
label = validation_generator.class_indices
indexlabel = dict((value, key) for key, value in label.items())

In [ ]:
predicts = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size, verbose=1)
predict_class = np.argmax(predicts, axis=1)
errors = np.where(predict_class != truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(truth,predict_class)

labels = []
for k,v in indexlabel.items():
    labels.append(v)
    
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, classes=labels,
                      title='Confusion matrix, without normalization')

In [ ]:
#and reports metrics with classification_report method
def predict_and_report(gen, model):
    y_true = []
    y_pred = []
    gen.reset()
    for img, label in gen:
        #get true labels for batch and store them
        y_true.extend([int(z[1]) for z in label])
        #Get predictions as probabilities
        batch_pred = model.predict_on_batch(img)
        #turn probabilities to class labels and store
        batch_pred = np.argmax(batch_pred, axis=1)
        y_pred.extend(batch_pred)
        #break loop
        if gen.batch_index == 0:
            break
            
    print('Accuracy:', accuracy_score(y_true, y_pred))
    print('Area Under the Receiver Operating Characteristic Curve:', roc_auc_score(y_true, y_pred)) #Area under the curve
    print(classification_report(y_true, y_pred))

In [ ]:
predict_and_report(train_generator, model)
predict_and_report(validation_generator, model)

In [ ]:
best_train_acc = max(result.history['acc'])
best_train_acc

In [ ]:
last_train_acc = result.history['acc'][-1]
last_train_acc

In [ ]:
results.append([train_dir, optimizer,learning_rate, batch_size, epochs, last_train_acc, best_train_acc, score[0], score[1]])

In [ ]:
import  pandas  as  pd
res = pd.DataFrame(results)
res.columns = ['data', 'optimizer','learning_rate','batch_size', 'epochs','last_train_accuracy', 'best_training_accuracy','test Score', 'test_accuracy']
res

In [ ]:
res.to_csv("Xception2-with.csv", sep=',', encoding='utf-8')